In [3]:
from problema.problema import Problema
import pulp as pu
import os
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px


In [5]:
problema = Problema(file='model_template_1205v2.xlsm')

Construyendo variables de planta


100%|██████████| 13/13 [00:00<00:00, 373.94it/s]


Construyendo componentes del modelo de planta


100%|██████████| 13/13 [00:00<00:00, 20.11it/s]


Construyendo componentes de material llegando


100%|██████████| 8/8 [00:00<00:00, 2963.91it/s]


Construyendo componentes de inventario en puerto


100%|██████████| 62/62 [00:00<00:00, 379.83it/s]


Agregando restricciones al modelo


100%|██████████| 10451/10451 [00:00<00:00, 163429.27it/s]


In [6]:
problema.solve(t_limit_minutes=20)

Cantidad de cpu: 15
gap: 1000000
t_limit: 20 minutos
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/luispinilla/Documents/source_code/bios_solver/env/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/0p/md4rzs_n7bg992nr5mzyl7_00000gn/T/2388f2cb43b046948e42bca6dee67b5d-pulp.mps sec 1200 allow 1000000 threads 15 timeMode elapsed branch printingOptions all solution /var/folders/0p/md4rzs_n7bg992nr5mzyl7_00000gn/T/2388f2cb43b046948e42bca6dee67b5d-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 10456 COLUMNS
At line 174354 RHS
At line 184806 BOUNDS
At line 207529 ENDATA
Problem MODEL has 10451 rows, 35928 columns and 87520 elements
Coin0008I MODEL read with 0 errors
seconds was changed from 1e+100 to 1200
allowableGap was changed from 1e-10 to 1e+06
threads was changed from 0 to 15
Option for timeMode changed from cpu to elapsed
Continuous objective value is 2.47574e+10 - 0.10 seconds


'Optimal'

In [7]:
plantas = problema.plantas
importaciones = problema.importaciones
transporte = problema.transporte
ingredientes = [nombre for nombre, ingrediente in problema.ingredientes.items()]
periodos = problema.periodos
plantas_df = problema.plantas_df

In [8]:
transporte_df = problema.reporte_transporte()

In [9]:
def reporte_puerto(problema)->pd.DataFrame:
    
    fields = {
        0: 'tipo',
        1: 'empresa',
        2: 'ingrediente',
        3: 'importacion',
        4: 'periodo'
    }

    df_dict = dict()
    df_dict['fecha'] = list()
    df_dict['inventario_kg'] = list()
    df_dict['costo_vencimiento_kg'] = list()
    df_dict['despachos_directos'] = list()
    df_dict['ingreso_bodega'] = list()

    for indice, titulo in fields.items():
        df_dict[titulo] = list()
    
    for importacion_name, importacion in problema.importaciones.items():
        for periodo, inventario_var in importacion.inventario_al_cierre.items():
            
            despachos = importacion.obtener_despachos_a_planta(periodo=periodo)
            despachos = sum([x.varValue for x in despachos if 'directo' in x.name])
            
            if periodo in importacion.variables_bodegaje.keys():
                bodegaje = importacion.variables_bodegaje[periodo].varValue
            else:
                bodegaje = 0.0            

            fecha = problema.periodos[periodo]
            campos = inventario_var.name.split('_')
            df_dict['fecha'].append(fecha)
            df_dict['inventario_kg'].append(inventario_var.varValue)
            df_dict['despachos_directos'].append(despachos)
            df_dict['ingreso_bodega'].append(bodegaje)
            
            if periodo in importacion.costo_vencimiento.keys():
                df_dict['costo_vencimiento_kg'].append(importacion.costo_vencimiento[periodo])
            else:
                df_dict['costo_vencimiento_kg'].append(0.0)

            for indice, titulo in fields.items():
                df_dict[titulo].append(campos[indice])

    df = pd.DataFrame(df_dict)

    df['costo_total_vencimiento'] = df['inventario_kg']*df['costo_vencimiento_kg']



    return df

In [10]:
puerto_df = reporte_puerto(problema=problema)
puerto_df

,fecha,inventario_kg,costo_vencimiento_kg,despachos_directos,ingreso_bodega,tipo,empresa,ingrediente,importacion,periodo,costo_total_vencimiento
0,2023-11-30,0.0,0.0,0,0.0,xip,contegral,gluten,DEVBULKDENIZ,0,0.0
1,2023-12-01,0.0,0.0,0,0.0,xip,contegral,gluten,DEVBULKDENIZ,1,0.0
2,2023-12-02,0.0,0.0,0,0.0,xip,contegral,gluten,DEVBULKDENIZ,2,0.0
3,2023-12-03,0.0,0.0,0,0.0,xip,contegral,gluten,DEVBULKDENIZ,3,0.0
4,2023-12-04,0.0,0.0,0,0.0,xip,contegral,gluten,DEVBULKDENIZ,4,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1917,2023-12-26,0.0,0.0,0,0.0,xip,finca,maiz,5653BREEZE,26,0.0
1918,2023-12-27,0.0,0.0,0,0.0,xip,finca,maiz,5653BREEZE,27,0.0
1919,2023-12-28,0.0,0.0,0,0.0,xip,finca,maiz,5653BREEZE,28,0.0
1920,2023-12-29,0.0,0.0,0,0.0,xip,finca,maiz,5653BREEZE,29,0.0


In [11]:
puerto_df.to_excel('borrame.xlsx')


In [12]:
barcos_df = problema.barcos_df
barcos_df

,empresa,operador,puerto,ingrediente,importacion,cantidad_kg,fecha_llegada,valor_kg,periodo,bodega,directo
0,contegral,trademar,baq,gluten,DEVBULKDENIZ,3500000,2023-12-13,2788.273427,13,12.925625,5.539554
1,finca,trademar,baq,gluten,DEVBULKDENIZ,1000000,2023-12-13,2788.273427,13,12.925625,5.539554
2,contegral,trademar,baq,forraje,DEVBULKDENIZ,2250000,2023-12-13,1312.513720,13,12.925625,5.539554
3,finca,trademar,baq,forraje,DEVBULKDENIZ,2250000,2023-12-13,1312.513720,13,12.925625,5.539554
4,contegral,opp,bun,maiz,0824MVCLLUZHOU,5500000,2023-12-14,976.757440,14,21.908600,9.389400
5,finca,opp,bun,maiz,5650MVCLLUZHOU,3700000,2023-12-14,976.757440,14,21.908600,9.389400
6,contegral,opp,bun,maiz,0823SERVETTE,12000000,2023-12-20,976.757440,20,21.908600,9.389400
7,finca,opp,bun,maiz,5649SERVETTE,6000000,2023-12-20,976.757440,20,21.908600,9.389400


In [13]:
impo.obtener_envios_a_bodega()

NameError: name 'impo' is not defined

In [ ]:
impo = problema.importaciones['contegral_compas_maiz_0765FEDERALIZUMI']

print()
for periodo, inv_var in impo.inventario_al_cierre.items():
    #print(periodo,inv_var.varValue)
    pass

In [ ]:
reporte_plantas_df = problema.reporte_planta()

In [ ]:
def dibujar_planta(df:pd.DataFrame, nombre_planta:str, ingrediente:str):
    
    temp = df[(df['planta']==nombre_planta)&(df['ingrediente']==ingrediente)]
    
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=temp['fecha'], # x-axis
        y=temp['inventario'], # y-axis
        mode='lines', # Connect data points with lines
        name='Inventario' # Name in the legend
    ))

    fig.add_trace(go.Scatter(
        x=temp['fecha'], # x-axis
        y=temp['capacidad'], # y-axis
        mode='lines', # Connect data points with lines
        name='Capacidad Almacenamiento' # Name in the legend
    ))


    fig.add_trace(go.Bar(
        x=temp['fecha'], 
        y=temp['llegada_bodega'],
        name='Despacho Bodega')
    )

    fig.add_trace(go.Bar(
        x=temp['fecha'], 
        y=temp['llegada_directa'],
        name='Despacho Directo')
    )

    fig.add_trace(go.Scatter(
        x=temp['fecha'], # x-axis
        y=temp['safety_stock'], # y-axis
        mode='lines', # Connect data points with lines
        name='Safety Stock' # Name in the legend
    ))

    fig.add_trace(go.Bar(
        x=temp['fecha'], # x-axis
        y=temp['llegada_programada'], # y-axis
        name='Llegadas Programada' # Name in the legend
    ))

    fig.add_trace(go.Scatter(
        x=temp['fecha'], # x-axis
        y=temp['backorder'], # y-axis
        mode='lines', # Connect data points with lines
        name='Backorder' # Name in the legend
    ))

    # Layout parameters
    fig.update_layout(
        title=f'Planta: {nombre_planta}: {ingrediente}', # Title
        xaxis_title='Fecha', # y-axis name
        yaxis_title='Kg', # x-axis name
        xaxis_tickangle=45,  # Set the x-axis label angle
        showlegend=True,     # Display the legend
        barmode='stack'
    )
    fig.show()

In [ ]:
for nombre, planta in problema.plantas.items(): 
    dibujar_planta(df=reporte_plantas_df, nombre_planta=nombre, ingrediente='frijol')